In [1]:
import numpy as np
import pandas as pd
import itertools
import folium
import time
import openrouteservice as ors
import sys
import urllib.parse
import pulp

In [8]:
ors_key = "5b3ce3597851110001cf624804ffaeec7cd246038d01eb4d3a32f633"
ors_boot = ors.Client(ors_key)

for i in range(1):
    pos_1 = "35.5212561" , "139.4921609"
    pos_2 = "35.196344" , "139.0247288"
    p1r = tuple(reversed(pos_1))
    p2r = tuple(reversed(pos_2))
    
    route_rslt = ors_boot.directions((p1r,p2r),profile = "driving-car")
    tfr_time = route_rslt["routes"][0]["summary"]["duration"]
tfr_time

4537.4

In [43]:
lim_time = 8*60*60
count_place = 8
place_remove = list(range(count_place))
for i in place_remove.copy():
    if i == 1:
        place_remove.remove(1)
    elif i == 0:
        place_remove.remove(0)


for vehicle_count in range(1,1+1):
    # 問題の宣言
    problem = pulp.LpProblem("CVRP", pulp.LpMaximize)

    # 決定変数
    x = [[[pulp.LpVariable("x%s_%s,%s"%(i,j,k), cat="Binary") if i != j else None for k in range(vehicle_count)]for j in range(count_place)] for i in range(count_place)]

    # 目的関数
    problem += pulp.lpSum(x[i][j][k] if i != j else 0
                          for k in range(vehicle_count) for j in range(count_place) for i in range (count_place))

    # 制約
    # (2)式，各顧客の場所に訪れるのは1台の車両で1度である
    for j in range(1, count_place):
        problem += pulp.lpSum(x[i][j][k] if i != j else 0 for i in range(count_place) for k in range(vehicle_count)) == 1 

    #(3)式, depotから出発して，depotに戻ってくる
    for k in range(vehicle_count):
        # デポを出発した運搬車が必ず 1つの顧客から訪問を開始することを保証する制約条件
        problem += pulp.lpSum(x[0][j][k] for j in range(1,count_place)) == 1
        # 必ず 1 つの顧客から運搬車がデポへ到着すること保証する制約条件
        problem += pulp.lpSum(x[i][1][k] for i in place_remove) == 1

    #(4)式, ある顧客の所に来る車両数と出る車両数が同じ
    for k in range(vehicle_count):
        for j in range(count_place):
            problem += pulp.lpSum(x[i][j][k] if i != j else 0 for i in range(count_place)) -  pulp.lpSum(x[j][i][k] for i in range(count_place)) == 0

    # #(5)式, 各車両において最大容量を超えない
    # for k in range(vehicle_count):
    #     problem += pulp.lpSum(df.demand[j] * x[i][j][k] if i != j else 0 for i in range(count_place) for j in range (1,count_place)) <= vehicle_capacity 


    #(6)式, 部分巡回路除去制約
    subtours = []
    for i in range(2,count_place):
         subtours += itertools.combinations(range(1,count_place), i)

    for s in subtours:
        problem += pulp.lpSum(x[i][j][k] if i !=j else 0 for i, j in itertools.permutations(s,2) for k in range(vehicle_count)) <= len(s) - 1
    
    print(problem)

CVRP:
MAXIMIZE
1*x0_1,0 + 1*x0_2,0 + 1*x0_3,0 + 1*x0_4,0 + 1*x0_5,0 + 1*x0_6,0 + 1*x0_7,0 + 1*x1_0,0 + 1*x1_2,0 + 1*x1_3,0 + 1*x1_4,0 + 1*x1_5,0 + 1*x1_6,0 + 1*x1_7,0 + 1*x2_0,0 + 1*x2_1,0 + 1*x2_3,0 + 1*x2_4,0 + 1*x2_5,0 + 1*x2_6,0 + 1*x2_7,0 + 1*x3_0,0 + 1*x3_1,0 + 1*x3_2,0 + 1*x3_4,0 + 1*x3_5,0 + 1*x3_6,0 + 1*x3_7,0 + 1*x4_0,0 + 1*x4_1,0 + 1*x4_2,0 + 1*x4_3,0 + 1*x4_5,0 + 1*x4_6,0 + 1*x4_7,0 + 1*x5_0,0 + 1*x5_1,0 + 1*x5_2,0 + 1*x5_3,0 + 1*x5_4,0 + 1*x5_6,0 + 1*x5_7,0 + 1*x6_0,0 + 1*x6_1,0 + 1*x6_2,0 + 1*x6_3,0 + 1*x6_4,0 + 1*x6_5,0 + 1*x6_7,0 + 1*x7_0,0 + 1*x7_1,0 + 1*x7_2,0 + 1*x7_3,0 + 1*x7_4,0 + 1*x7_5,0 + 1*x7_6,0 + 0
SUBJECT TO
_C1: x0_1,0 + x2_1,0 + x3_1,0 + x4_1,0 + x5_1,0 + x6_1,0 + x7_1,0 = 1

_C2: x0_2,0 + x1_2,0 + x3_2,0 + x4_2,0 + x5_2,0 + x6_2,0 + x7_2,0 = 1

_C3: x0_3,0 + x1_3,0 + x2_3,0 + x4_3,0 + x5_3,0 + x6_3,0 + x7_3,0 = 1

_C4: x0_4,0 + x1_4,0 + x2_4,0 + x3_4,0 + x5_4,0 + x6_4,0 + x7_4,0 = 1

_C5: x0_5,0 + x1_5,0 + x2_5,0 + x3_5,0 + x4_5,0 + x6_5,0 + x7_5,0 = 1

_